In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

#Hard Margin

### Data

In [ ]:
X = np.array([[-1, 2], [1, 1], [-1, 0], [0, -1]])
y = np.array([1, 1, -1, -1])
y = y.reshape(y.shape[0], 1)

### CVXOPT

#### Primal

In [ ]:
N, M = X.shape
M += 1
K = np.eye(M)
K[M-1, M-1] = 0
K = K * 1.
K

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [ ]:
p = np.zeros((M, 1))
p

array([[0.],
       [0.],
       [0.]])

In [ ]:
G = np.concatenate((X, np.ones((N, 1))), axis = 1) * -np.repeat(y, M, axis=1)
G

array([[ 1., -2., -1.],
       [-1., -1., -1.],
       [-1.,  0.,  1.],
       [ 0., -1.,  1.]])

In [ ]:
h = -np.ones((N, 1))
h

array([[-1.],
       [-1.],
       [-1.],
       [-1.]])

In [ ]:
K = cvxopt_matrix(K)
p = cvxopt_matrix(p)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)


solution = cvxopt_solvers.qp(K, p, G, h)
wb = np.array(solution['x'])
wb

     pcost       dcost       gap    pres   dres
 0:  3.0728e-01  5.4665e-01  9e-01  1e+00  3e-16
 1:  4.6696e-01  5.9739e-01  4e-02  2e-01  2e-16
 2:  6.2209e-01  6.2429e-01  2e-03  4e-03  4e-15
 3:  6.2497e-01  6.2499e-01  2e-05  4e-05  6e-16
 4:  6.2500e-01  6.2500e-01  2e-07  4e-07  5e-16
 5:  6.2500e-01  6.2500e-01  2e-09  4e-09  2e-16
Optimal solution found.


array([[ 0.5],
       [ 1. ],
       [-0.5]])

#### Dual


In [ ]:
N, M = X.shape
KGram = (X @ X.T)
KGram 

array([[ 5,  1,  1, -2],
       [ 1,  2, -1, -1],
       [ 1, -1,  1,  0],
       [-2, -1,  0,  1]])

In [ ]:
Y = y @ y.T
Y

array([[ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1]])

In [ ]:
K = KGram * Y * 1.
K

array([[ 5.,  1., -1.,  2.],
       [ 1.,  2.,  1.,  1.],
       [-1.,  1.,  1.,  0.],
       [ 2.,  1.,  0.,  1.]])

In [ ]:
p = -np.ones((N, 1))
p

array([[-1.],
       [-1.],
       [-1.],
       [-1.]])

In [ ]:
G = -np.eye(N)
G

array([[-1., -0., -0., -0.],
       [-0., -1., -0., -0.],
       [-0., -0., -1., -0.],
       [-0., -0., -0., -1.]])

In [ ]:
h = np.zeros((N, 1))
h

array([[0.],
       [0.],
       [0.],
       [0.]])

In [ ]:
A = y.T * 1.
A

array([[ 1.,  1., -1., -1.]])

In [ ]:
b = np.zeros((1, 1))
b

array([[0.]])

In [ ]:
#Solving
K = cvxopt_matrix(K)
p = cvxopt_matrix(p)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)
A = cvxopt_matrix(A)
b = cvxopt_matrix(b)

solution = cvxopt_solvers.qp(K, p, G, h, A, b)
alpha = np.array(solution['x'])
alpha

     pcost       dcost       gap    pres   dres
 0: -5.4665e-01 -1.4874e+00  9e-01  5e-17  1e+00
 1: -5.9739e-01 -6.3518e-01  4e-02  6e-17  2e-01
 2: -6.2429e-01 -6.2646e-01  2e-03  2e-16  4e-03
 3: -6.2499e-01 -6.2501e-01  2e-05  2e-16  4e-05
 4: -6.2500e-01 -6.2500e-01  2e-07  2e-16  4e-07
 5: -6.2500e-01 -6.2500e-01  2e-09  3e-18  4e-09
Optimal solution found.


array([[3.74999998e-01],
       [2.50000000e-01],
       [6.24999997e-01],
       [1.40673228e-09]])

In [ ]:
S_filter = (alpha > 1e-4)
alpha_S  = alpha[S_filter].reshape(-1, 1)
alpha_S

array([[0.375],
       [0.25 ],
       [0.625]])

In [ ]:
y_S = y[S_filter].reshape(-1, 1)
y_S

array([[ 1],
       [ 1],
       [-1]])

In [ ]:
S_filter

array([[ True],
       [ True],
       [ True],
       [False]])

In [ ]:
X_S = X[S_filter.flatten(), :]
X_S

array([[-1,  2],
       [ 1,  1],
       [-1,  0]])

In [ ]:
w = (y_S * alpha_S).T @ X_S
w

array([[0.5, 1. ]])

In [ ]:
N_S = X_S.shape[0]
K_SS = X_S @ X_S.T
K_SS

array([[ 5,  1,  1],
       [ 1,  2, -1],
       [ 1, -1,  1]])

In [ ]:
one_vec = np.ones((N_S, 1))
one_vec

array([[1.],
       [1.],
       [1.]])

In [ ]:
b = ((y_S - K_SS @ (alpha_S * y_S)).T @ one_vec) / N_S
b

array([[-0.5]])

# Soft Margin

In [ ]:
C = 10
X = np.array([[3, 4], [1, 4], [2, 3], [6, -1], [7, -1], [5, -3], [2, 4]])
y = np.array([1, 1, 1, -1, -1, -1, -1]).reshape(-1, 1)

In [ ]:
KGram = X @ X.T
KGram

array([[25, 19, 18, 14, 17,  3, 22],
       [19, 17, 14,  2,  3, -7, 18],
       [18, 14, 13,  9, 11,  1, 16],
       [14,  2,  9, 37, 43, 33,  8],
       [17,  3, 11, 43, 50, 38, 10],
       [ 3, -7,  1, 33, 38, 34, -2],
       [22, 18, 16,  8, 10, -2, 20]])

In [ ]:
Y = y @ y.T
Y

array([[ 1,  1,  1, -1, -1, -1, -1],
       [ 1,  1,  1, -1, -1, -1, -1],
       [ 1,  1,  1, -1, -1, -1, -1],
       [-1, -1, -1,  1,  1,  1,  1],
       [-1, -1, -1,  1,  1,  1,  1],
       [-1, -1, -1,  1,  1,  1,  1],
       [-1, -1, -1,  1,  1,  1,  1]])

In [ ]:
K = KGram * Y * 1.
K

array([[ 25.,  19.,  18., -14., -17.,  -3., -22.],
       [ 19.,  17.,  14.,  -2.,  -3.,   7., -18.],
       [ 18.,  14.,  13.,  -9., -11.,  -1., -16.],
       [-14.,  -2.,  -9.,  37.,  43.,  33.,   8.],
       [-17.,  -3., -11.,  43.,  50.,  38.,  10.],
       [ -3.,   7.,  -1.,  33.,  38.,  34.,  -2.],
       [-22., -18., -16.,   8.,  10.,  -2.,  20.]])

In [ ]:
N, M = X.shape
p = -np.ones((N, 1))
p

array([[-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.]])

In [ ]:
G = np.vstack((-np.eye(N), np.eye(N)))
G

array([[-1., -0., -0., -0., -0., -0., -0.],
       [-0., -1., -0., -0., -0., -0., -0.],
       [-0., -0., -1., -0., -0., -0., -0.],
       [-0., -0., -0., -1., -0., -0., -0.],
       [-0., -0., -0., -0., -1., -0., -0.],
       [-0., -0., -0., -0., -0., -1., -0.],
       [-0., -0., -0., -0., -0., -0., -1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [ ]:
A = y.T * 1.
A

array([[ 1.,  1.,  1., -1., -1., -1., -1.]])

In [ ]:
h = np.hstack((np.zeros(N), np.ones(N) * C)).reshape(-1, 1)
h

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [10.],
       [10.],
       [10.],
       [10.],
       [10.],
       [10.],
       [10.]])

In [ ]:
b = np.zeros((1,1))
b

array([[0.]])

In [ ]:
K = cvxopt_matrix(K)
p = cvxopt_matrix(p)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)
A = cvxopt_matrix(A)
b = cvxopt_matrix(b)

solution = cvxopt_solvers.qp(K, p, G, h, A, b)
alpha = np.array(solution['x'])
alpha

     pcost       dcost       gap    pres   dres
 0: -1.5556e+01 -2.5999e+02  3e+02  1e-01  2e-14
 1: -1.8050e+01 -3.9497e+01  2e+01  4e-03  1e-14
 2: -2.1437e+01 -2.3412e+01  2e+00  3e-04  2e-14
 3: -2.2496e+01 -2.2997e+01  5e-01  3e-05  1e-14
 4: -2.2561e+01 -2.2568e+01  6e-03  3e-07  2e-14
 5: -2.2562e+01 -2.2563e+01  6e-05  3e-09  4e-14
 6: -2.2562e+01 -2.2563e+01  6e-07  3e-11  2e-14
Optimal solution found.


array([[4.99999980e+00],
       [1.52461667e-08],
       [6.31250013e+00],
       [1.31249982e+00],
       [2.36607341e-08],
       [1.32909858e-07],
       [9.99999997e+00]])

In [ ]:
S_filter = (alpha > 1e-4)
alpha_S  = alpha[S_filter].reshape(-1, 1)
alpha_S

array([[4.9999998 ],
       [6.31250013],
       [1.31249982],
       [9.99999997]])

In [ ]:
M_filter = (alpha_S < (C - 1e-4))
alpha_M = alpha_S[M_filter].reshape(-1, 1)
alpha_M

array([[4.9999998 ],
       [6.31250013],
       [1.31249982]])

In [ ]:
y_S = y[S_filter].reshape(-1, 1)
y_S

array([[ 1],
       [ 1],
       [-1],
       [-1]])

In [ ]:
y_M = y_S[M_filter].reshape(-1, 1)
y_M

array([[ 1],
       [ 1],
       [-1]])

In [ ]:
X_S = X[S_filter.flatten(), :]
X_S

array([[ 3,  4],
       [ 2,  3],
       [ 6, -1],
       [ 2,  4]])

In [ ]:
X_M = X_S[M_filter.flatten(), :]
X_M

array([[ 3,  4],
       [ 2,  3],
       [ 6, -1]])

In [ ]:
w = (y_S * alpha_S).T @ X_S
w

array([[-0.24999919,  0.24999953]])

In [ ]:
N_M = X_M.shape[0]
K_MS = X_M @ X_S.T
K_MS

array([[25, 18, 14, 22],
       [18, 13,  9, 16],
       [14,  9, 37,  8]])

In [ ]:
b = ((y_M - K_MS @ (alpha_S * y_S)).T @ np.ones((N_M, 1))) / N_M
b

array([[0.74999799]])